In [13]:
import pandas as pd
import numpy as np
import pandas_profiling as pf
import matplotlib.pyplot as plt
from sklearn.preprocessing import OrdinalEncoder,LabelEncoder
from sklearn.model_selection import GridSearchCV, cross_val_score, train_test_split
from sklearn.metrics import (
    ConfusionMatrixDisplay,
    classification_report,
    confusion_matrix,
)
import xgboost as xgb
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.ensemble import GradientBoostingClassifier

In [14]:
df = pd.read_csv("first general DATA(KNN).csv",)
df.head()

,HeartDisease,BMI,Smoking,AlcoholDrinking,Stroke,DiffWalking,Sex,AgeCategory,Diabetic,PhysicalActivity,Asthma,HighBP,CholCheck
0,No,underweight,Yes,No,No,No,Female,55-59,Yes,Yes,Yes,No,Yes
1,No,healthy weight,No,No,Yes,No,Female,80 or older,No,Yes,No,Yes,Yes
2,No,overweight,Yes,No,No,No,Male,65-69,Yes,Yes,Yes,Yes,Yes
3,No,healthy weight,No,No,No,No,Female,75-79,No,No,No,No,Yes
4,No,healthy weight,No,No,No,Yes,Female,40-44,No,Yes,No,No,No


In [15]:
target = 'HeartDisease'
X = df.drop(columns=[target])
y = df[target]

In [16]:
oe = OrdinalEncoder()
X = oe.fit_transform(X)
X

array([[6., 1., 0., ..., 1., 0., 1.],
       [0., 0., 0., ..., 0., 1., 1.],
       [3., 1., 0., ..., 1., 1., 1.],
       ...,
       [3., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 1., 1.],
       [3., 0., 0., ..., 0., 1., 1.]])

In [17]:
le = LabelEncoder()
y = le.fit_transform(y)
y

array([0, 0, 0, ..., 0, 0, 1])

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

X_train shape: (458780, 12)
y_train shape: (458780,)
X_test shape: (114695, 12)
y_test shape: (114695,)


In [19]:
clf = GradientBoostingClassifier(max_depth=8,n_estimators=700,subsample= 0.8,verbose=1,random_state=42)

In [20]:
acc_baseline = 1 - y_train.mean()
print("Baseline Accuracy:", round(acc_baseline, 4))

Baseline Accuracy: 0.9105


In [21]:
weights = np.zeros(len(y_train))
weights[y_train==0]=1
weights[y_train==1]=4

In [22]:
clf.fit(X_train, y_train,weights)

      Iter       Train Loss      OOB Improve   Remaining Time 
         1           1.1343           0.0554           11.58m
         2           1.0887           0.0438           12.17m
         3           1.0508           0.0357           11.92m
         4           1.0210           0.0295           11.99m
         5           0.9942           0.0245           11.64m
         6           0.9723           0.0204           10.74m
         7           0.9562           0.0172           10.10m
         8           0.9417           0.0149            9.65m
         9           0.9283           0.0122            9.26m
        10           0.9168           0.0106            9.18m
        20           0.8563           0.0028            9.57m
        30           0.8328           0.0006            8.52m
        40           0.8248           0.0004            7.74m
        50           0.8179          -0.0001            7.37m
        60           0.8130          -0.0000            7.03m
       

GradientBoostingClassifier(max_depth=8, n_estimators=700, random_state=42,
                           subsample=0.8, verbose=1)

In [23]:
class_report = classification_report(y_test, clf.predict(X_test))

In [24]:
print(class_report)

              precision    recall  f1-score   support

           0       0.96      0.90      0.93    104478
           1       0.36      0.58      0.44     10217

    accuracy                           0.87    114695
   macro avg       0.66      0.74      0.68    114695
weighted avg       0.90      0.87      0.88    114695



In [25]:
ConfusionMatrixDisplay.from_estimator(clf,X_test,y_test)
plt.savefig("5-1.png", dpi=100)

In [26]:
import pickle
pickle.dump(clf, open("./model-final.pkl", "wb"))